Import packages and libraries

In [ ]:
import cv2
import pandas as pd
import ast
import shutil
import os
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
%ls

allImageFolder/  ObjectDetection.ipynb  train.csv
F2_score.ipynb   Preprocessing.ipynb    Visualize.ipynb


Read train.csv file and build data frame to extract annotation information for each image

In [ ]:
df = pd.read_csv('train.csv')
df['image_path']  = df.apply(lambda row: 'train_images/video_' + str(row.video_id) + '/' + str(row.video_frame) + '.jpg', axis=1)
df['new_image_path'] = df.apply(lambda row: 'allImageFolder/' + str(row.image_id) + '.jpg', axis=1)
df.iloc[100:105]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,image_path,new_image_path
100,0,40258,100,100,0-100,"[{'x': 276, 'y': 631, 'width': 116, 'height': ...",train_images/video_0/100.jpg,allImageFolder/0-100.jpg
101,0,40258,101,101,0-101,"[{'x': 266, 'y': 659, 'width': 116, 'height': ...",train_images/video_0/101.jpg,allImageFolder/0-101.jpg
102,0,40258,102,102,0-102,"[{'x': 263, 'y': 680, 'width': 116, 'height': ...",train_images/video_0/102.jpg,allImageFolder/0-102.jpg
103,0,40258,103,103,0-103,"[{'x': 642, 'y': 49, 'width': 20, 'height': 21}]",train_images/video_0/103.jpg,allImageFolder/0-103.jpg
104,0,40258,104,104,0-104,"[{'x': 642, 'y': 46, 'width': 22, 'height': 22}]",train_images/video_0/104.jpg,allImageFolder/0-104.jpg


In [ ]:
df['num_box'] = df.apply(lambda row: len(ast.literal_eval(row.annotations)), axis=1)
df

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,image_path,new_image_path,num_box
0,0,40258,0,0,0-0,[],train_images/video_0/0.jpg,allImageFolder/0-0.jpg,0
1,0,40258,1,1,0-1,[],train_images/video_0/1.jpg,allImageFolder/0-1.jpg,0
2,0,40258,2,2,0-2,[],train_images/video_0/2.jpg,allImageFolder/0-2.jpg,0
3,0,40258,3,3,0-3,[],train_images/video_0/3.jpg,allImageFolder/0-3.jpg,0
4,0,40258,4,4,0-4,[],train_images/video_0/4.jpg,allImageFolder/0-4.jpg,0
...,...,...,...,...,...,...,...,...,...
23496,2,29859,10755,2983,2-10755,[],train_images/video_2/10755.jpg,allImageFolder/2-10755.jpg,0
23497,2,29859,10756,2984,2-10756,[],train_images/video_2/10756.jpg,allImageFolder/2-10756.jpg,0
23498,2,29859,10757,2985,2-10757,[],train_images/video_2/10757.jpg,allImageFolder/2-10757.jpg,0
23499,2,29859,10758,2986,2-10758,[],train_images/video_2/10758.jpg,allImageFolder/2-10758.jpg,0


There are 23501 images in total.

In [ ]:
len(df) 

23501

Display the ratio of image with bounding boxes and without bounding boxes

In [ ]:
data = (df.num_box>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

No BBox: 79.07% | With BBox: 20.93%


There are 4919 images with bounding boxes

In [ ]:
df = df.query('num_box>0')
len(df)

4919

In [ ]:
old_image_paths = df.image_path.tolist()
new_image_paths = df.new_image_path.tolist()
for old, new in zip(old_image_paths, new_image_paths):
    shutil.copy(old, new)

Set value for width and height of each image and add label 'starfish' 

In [ ]:
df['width'] = 1280
df['height'] = 720
df['label'] = 'starfish'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df_train = df[:2500]
df_val = df[2500:int(len(df)*0.8)]
df_test = df[int(len(df)*0.8):]
df_train = df_train.sample(n=160)
# df_val = df_train.sample(n=20)
df_test = df_test.sample(n=5)

Data frame for trainning set

In [ ]:
df_train

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,image_path,new_image_path,num_box,width,height,label
3095,0,8399,5018,1280,0-5018,"[{'x': 569, 'y': 219, 'width': 65, 'height': 51}]",train_images/video_0/5018.jpg,allImageFolder/0-5018.jpg,1,1280,720,starfish
5702,0,53708,9879,994,0-9879,"[{'x': 828, 'y': 519, 'width': 35, 'height': 3...",train_images/video_0/9879.jpg,allImageFolder/0-9879.jpg,2,1280,720,starfish
2550,0,8399,4473,735,0-4473,"[{'x': 184, 'y': 218, 'width': 42, 'height': 18}]",train_images/video_0/4473.jpg,allImageFolder/0-4473.jpg,1,1280,720,starfish
2645,0,8399,4568,830,0-4568,"[{'x': 168, 'y': 90, 'width': 50, 'height': 43...",train_images/video_0/4568.jpg,allImageFolder/0-4568.jpg,4,1280,720,starfish
2552,0,8399,4475,737,0-4475,"[{'x': 169, 'y': 230, 'width': 44, 'height': 19}]",train_images/video_0/4475.jpg,allImageFolder/0-4475.jpg,1,1280,720,starfish
...,...,...,...,...,...,...,...,...,...,...,...,...
1757,0,59337,2330,479,0-2330,"[{'x': 300, 'y': 526, 'width': 50, 'height': 44}]",train_images/video_0/2330.jpg,allImageFolder/0-2330.jpg,1,1280,720,starfish
2596,0,8399,4519,781,0-4519,"[{'x': 56, 'y': 616, 'width': 78, 'height': 58}]",train_images/video_0/4519.jpg,allImageFolder/0-4519.jpg,1,1280,720,starfish
115,0,40258,115,115,0-115,"[{'x': 653, 'y': 8, 'width': 25, 'height': 26}]",train_images/video_0/115.jpg,allImageFolder/0-115.jpg,1,1280,720,starfish
270,0,40258,270,270,0-270,"[{'x': 1012, 'y': 436, 'width': 41, 'height': ...",train_images/video_0/270.jpg,allImageFolder/0-270.jpg,1,1280,720,starfish


Data frame for test set

In [ ]:
df_test

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,image_path,new_image_path,num_box,width,height,label
20287,2,22643,6385,1022,2-6385,"[{'x': 108, 'y': 313, 'width': 33, 'height': 33}]",train_images/video_2/6385.jpg,allImageFolder/2-6385.jpg,1,1280,720,starfish
19348,2,22643,5446,83,2-5446,"[{'x': 116, 'y': 439, 'width': 59, 'height': 55}]",train_images/video_2/5446.jpg,allImageFolder/2-5446.jpg,1,1280,720,starfish
19662,2,22643,5760,397,2-5760,"[{'x': 578, 'y': 336, 'width': 64, 'height': 5...",train_images/video_2/5760.jpg,allImageFolder/2-5760.jpg,15,1280,720,starfish
12839,1,60754,9274,858,1-9274,"[{'x': 343, 'y': 221, 'width': 43, 'height': 2...",train_images/video_1/9274.jpg,allImageFolder/1-9274.jpg,4,1280,720,starfish
20731,2,29859,7990,218,2-7990,"[{'x': 416, 'y': 486, 'width': 36, 'height': 34}]",train_images/video_2/7990.jpg,allImageFolder/2-7990.jpg,1,1280,720,starfish


Pre-processing function to convert image to grayscale then perform contrast stretching.

In [ ]:
def contrastStretching(img):
  Olist = []
  for i in range(3):
    a=0
    b=255
    c=img[:,:,i].min()
    d=img[:,:,i].max()
    O = (img[:,:,i] - c) * ((b-a)/(d-c)) + a
    O = O.astype(int)
    Olist.append(O)
  outputimg = cv2.merge(Olist)
  return outputimg

Copy images to directories without preprocessing

In [ ]:
Path("train").mkdir(parents=True, exist_ok=True)
# Path("validate").mkdir(parents=True, exist_ok=True)
Path("test").mkdir(parents=True, exist_ok=True)
for i in range(len(df_train)):
    shutil.copy(df_train['new_image_path'].iloc[i],'./train')
for i in range(len(df_test)):
    shutil.copy(df_test['new_image_path'].iloc[i],'./test')
# for i in range(len(df_val)):
#     shutil.copy(df_val['new_image_path'].iloc[i],'./validate')

Copy images to directories with preprocessing 



In [ ]:
img_list = []
Path("train2").mkdir(parents=True, exist_ok=True)
trainImage = os.listdir('train')
print(trainImage)
for i in range(len(trainImage)):
  img = cv2.imread('train/'+trainImage[i])
  img = contrastStretching(img)
  cv2.imwrite("train2/"+trainImage[i],img)


['0-5018.jpg', '0-9879.jpg', '0-4473.jpg', '0-4568.jpg', '0-4475.jpg', '0-2342.jpg', '1-485.jpg', '0-4541.jpg', '0-9917.jpg', '0-63.jpg', '0-9652.jpg', '0-9775.jpg', '0-12307.jpg', '0-9326.jpg', '0-4502.jpg', '1-4081.jpg', '0-4769.jpg', '0-122.jpg', '0-9367.jpg', '0-4714.jpg', '0-1030.jpg', '0-4755.jpg', '0-1556.jpg', '0-977.jpg', '0-9680.jpg', '0-8943.jpg', '0-9614.jpg', '1-3896.jpg', '0-55.jpg', '0-264.jpg', '0-9638.jpg', '0-9513.jpg', '0-60.jpg', '0-9533.jpg', '0-4846.jpg', '0-9708.jpg', '0-4678.jpg', '1-4095.jpg', '0-4245.jpg', '0-9754.jpg', '0-4844.jpg', '0-9740.jpg', '0-4682.jpg', '1-3967.jpg', '0-9246.jpg', '0-9644.jpg', '0-9937.jpg', '0-9747.jpg', '0-4607.jpg', '0-11881.jpg', '0-4624.jpg', '1-484.jpg', '0-11909.jpg', '0-12289.jpg', '0-4706.jpg', '0-368.jpg', '0-1893.jpg', '0-4210.jpg', '0-1922.jpg', '0-9799.jpg', '0-4275.jpg', '0-4262.jpg', '0-4583.jpg', '0-4559.jpg', '0-9553.jpg', '0-5721.jpg', '0-12254.jpg', '0-117.jpg', '0-4803.jpg', '0-268.jpg', '0-11919.jpg', '0-12276.jpg'

In [ ]:
img_list = []
Path("test2").mkdir(parents=True, exist_ok=True)
testImage = os.listdir('test')
print(testImage)
for i in range(len(testImage)):
  img = cv2.imread('test/'+testImage[i])
  img = contrastStretching(img)
  cv2.imwrite("test2/"+trainImage[i],img)

['2-6385.jpg', '2-5446.jpg', '2-5760.jpg', '1-9274.jpg', '2-7990.jpg']


Save training/validation set to csv file to be feed into model

In [ ]:
header = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax', 'image_id']
train_csv_df = pd.DataFrame(columns=header)
for i in range(len(df_train)):
    annot = ast.literal_eval(df_train['annotations'].iloc[i])[0]
    dfList = [str(df_train['image_id'].iloc[i]+".jpg"),df_train['width'].iloc[i],df_train['height'].iloc[i],df_train['label'].iloc[i],annot['x'],annot['y'],annot['x']+annot['width'],annot['y']+annot['height'],int(i)]
    train_csv_df.loc[len(train_csv_df)] = dfList
train_csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id
0,0-5018.jpg,1280,720,starfish,569,219,634,270,0
1,0-9879.jpg,1280,720,starfish,828,519,863,552,1
2,0-4473.jpg,1280,720,starfish,184,218,226,236,2
3,0-4568.jpg,1280,720,starfish,168,90,218,133,3
4,0-4475.jpg,1280,720,starfish,169,230,213,249,4
...,...,...,...,...,...,...,...,...,...
155,0-2330.jpg,1280,720,starfish,300,526,350,570,155
156,0-4519.jpg,1280,720,starfish,56,616,134,674,156
157,0-115.jpg,1280,720,starfish,653,8,678,34,157
158,0-270.jpg,1280,720,starfish,1012,436,1053,477,158


In [ ]:
train_csv_df.to_csv('train_labels.csv', index=False)

In [ ]:
val_csv_df = pd.DataFrame(columns=header)
for i in range(len(df_val)):
    annot = ast.literal_eval(df_val['annotations'].iloc[i])[0]
    dfList = [str(df_val['image_id'].iloc[i]+".jpg"),df_val['width'].iloc[i],df_val['height'].iloc[i],df_val['label'].iloc[i],annot['x'],annot['y'],annot['x']+annot['width'],annot['y']+annot['height'],int(i)]
    val_csv_df.loc[len(val_csv_df)] = dfList
val_csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id
0,1-4121.jpg,1280,720,starfish,897,88,951,127,0
1,1-4122.jpg,1280,720,starfish,495,124,527,147,1
2,1-4123.jpg,1280,720,starfish,499,135,530,157,2
3,1-4124.jpg,1280,720,starfish,918,117,975,158,3
4,1-4125.jpg,1280,720,starfish,928,134,981,166,4
...,...,...,...,...,...,...,...,...,...
1430,1-9142.jpg,1280,720,starfish,490,140,528,181,1430
1431,1-9143.jpg,1280,720,starfish,489,150,527,191,1431
1432,1-9144.jpg,1280,720,starfish,488,159,526,200,1432
1433,1-9145.jpg,1280,720,starfish,491,169,529,210,1433


In [ ]:
val_csv_df.to_csv('val_labels.csv', index=False)

Save test set information to csv to be used for extracting groundtruth information

In [ ]:
test_csv_df = pd.DataFrame(columns=header)
for i in range(len(df_test)):
    annot = ast.literal_eval(df_test['annotations'].iloc[i])[0]
    dfList = [str(df_test['image_id'].iloc[i]+".jpg"),df_test['width'].iloc[i],df_test['height'].iloc[i],df_test['label'].iloc[i],annot['x'],annot['y'],annot['x']+annot['width'],annot['y']+annot['height'],int(i)]
    test_csv_df.loc[len(test_csv_df)] = dfList
test_csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id
0,2-6385.jpg,1280,720,starfish,108,313,141,346,0
1,2-5446.jpg,1280,720,starfish,116,439,175,494,1
2,2-5760.jpg,1280,720,starfish,578,336,642,388,2
3,1-9274.jpg,1280,720,starfish,343,221,386,248,3
4,2-7990.jpg,1280,720,starfish,416,486,452,520,4


In [ ]:
test_csv_df.to_csv('test_labels.csv', index=False)